# Exploratory Data Analysis

In [1]:
import sqlite3
import pandas as pd

In [2]:
# creating connection
conn=sqlite3.connect('myntra_sales.db')

In [3]:
# checking tables present in the database
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table'", conn)

In [4]:
tables=tables['name'].tolist()

In [5]:
# drop the unneccesary table 
for item in ['O', 'Online Retail.csv']:
    if item in tables:
        tables.remove(item)


In [6]:
# exploring what type of data available in the tables
for table in tables:
    print('-'*50, f'{table}', '-'*50)
    try:
        query = f'SELECT count(*) as count FROM "{table}"'
        count = pd.read_sql_query(query, conn)['count'].values[0]
        print('Count of records:', count)

        # Preview top 5 rows
        preview = pd.read_sql_query(f'SELECT * FROM "{table}" LIMIT 5', conn)
        display(preview)

    except Exception as e:
        print(f"❌ Error accessing table '{table}': {e}")

-------------------------------------------------- Clean_Retail_Data --------------------------------------------------
Count of records: 0


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Total_Quantity,Total_UnitPrice,...,Year,Month,Quarter,Weekend,InvoiceHour,DayOfWeek,IsWeekend,PartOfDay,Avg_Price_Per_Item,Items_Per_Invoice


-------------------------------------------------- Online_Retail.csv --------------------------------------------------
Count of records: 344393


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


-------------------------------------------------- vendor_sales_summary --------------------------------------------------
Count of records: 328610


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Total_Quantity,Total_UnitPrice,...,Year,Month,Quarter,Weekend,InvoiceHour,DayOfWeek,IsWeekend,PartOfDay,Avg_Price_Per_Item,Items_Per_Invoice
0,550193,PADS,PADS TO MATCH ALL CUSHIONS,1,2011-04-15 09:27:00,0.001,13952.0,United Kingdom,1,0.001,...,2011,4,2,4,9,4,0,Afternoon,0.001,688
1,561226,PADS,PADS TO MATCH ALL CUSHIONS,1,2011-07-26 10:13:00,0.001,15618.0,United Kingdom,1,0.001,...,2011,7,3,1,10,1,0,Afternoon,0.001,96
2,568200,PADS,PADS TO MATCH ALL CUSHIONS,1,2011-09-25 14:58:00,0.001,16198.0,United Kingdom,1,0.001,...,2011,9,3,6,14,6,1,Evening,0.001,117
3,568375,BANK CHARGES,Bank Charges,1,2011-09-26 17:01:00,0.001,13405.0,United Kingdom,1,0.001,...,2011,9,3,0,17,0,0,Evening,0.001,1
4,578757,16216,LETTER SHAPE PENCIL SHARPENER,1,2011-11-25 11:41:00,0.060,12748.0,United Kingdom,1,0.060,...,2011,11,4,4,11,4,0,Afternoon,0.060,204


-------------------------------------------------- myntra_sales_summary --------------------------------------------------
Count of records: 328610


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Total_Quantity,Total_Revenue,Year,Month,Quarter,Weekend,InvoiceHour,DayOfWeek,IsWeekend,PartOfDay,Avg_Price_Per_Item,Items_Per_Invoice
0,550193,PADS,pads_to_match_all_cushions,1,2011-04-15 09:27:00,0.001,13952.0,United Kingdom,1,0.001,2011,April,2,4,9,4,0,Afternoon,0.001,688
1,561226,PADS,pads_to_match_all_cushions,1,2011-07-26 10:13:00,0.001,15618.0,United Kingdom,1,0.001,2011,July,3,1,10,1,0,Afternoon,0.001,96
2,568200,PADS,pads_to_match_all_cushions,1,2011-09-25 14:58:00,0.001,16198.0,United Kingdom,1,0.001,2011,September,3,6,14,6,1,Evening,0.001,117
3,568375,BANK CHARGES,bank_charges,1,2011-09-26 17:01:00,0.001,13405.0,United Kingdom,1,0.001,2011,September,3,0,17,0,0,Evening,0.001,1
4,578757,16216,letter_shape_pencil_sharpener,1,2011-11-25 11:41:00,0.060,12748.0,United Kingdom,1,0.060,2011,November,4,4,11,4,0,Afternoon,0.060,204


# Summary Statistics

In [7]:
# Total count of unique_customers and products within the dataframe
pd.read_sql_query('''
    SELECT
        count(*) as Total_records ,
        count(Distinct CustomerID) as Unique_Customer,
        count(Distinct Description) as Unique_Products
    from "Online_Retail.csv"
    where CustomerID IS NOT NULL;
''',conn)

,Total_records,Unique_Customer,Unique_Products
0,344393,4222,3598


# Top 10 Products by Quantity Sold

In [8]:
# Top 10 products by quantity sold
pd.read_sql_query("""
    SELECT 
        Description,
        Sum(Quantity) as Total_Quantity
    from "Online_Retail.csv"
    Where CustomerID IS NOT NULL
    Group BY Description
    Order By Total_Quantity DESC
    Limit 10;
""",conn)


,Description,Total_Quantity
0,PACK OF 72 RETROSPOT CAKE CASES,14994
1,ASSORTED COLOUR BIRD ORNAMENT,13641
2,JUMBO BAG RED RETROSPOT,12053
3,WHITE HANGING HEART T-LIGHT HOLDER,10697
4,LUNCH BAG RED RETROSPOT,9695
5,JAM MAKING SET PRINTED,8627
6,PACK OF 60 PINK PAISLEY CAKE CASES,8527
7,60 TEATIME FAIRY CAKE CASES,8396
8,VICTORIAN GLASS HANGING T-LIGHT,7538
9,RED RETROSPOT WRAP,7472


# Top 10 Customer by Spending

In [9]:
# Top 10 customer by 
pd.read_sql_query("""
    SELECT 
        CustomerID,
        Sum(Quantity * UnitPrice) as Total_Revenue
    from "Online_Retail.csv"
    Where CustomerID IS NOT NULL
    Group BY CustomerID
    Order By Total_Revenue DESC
    Limit 10;
""",conn)

,CustomerID,Total_Revenue
0,14911.0,83258.12
1,13089.0,36053.63
2,14096.0,36018.24
3,17841.0,32307.38
4,14298.0,24387.53
5,12748.0,20889.79
6,14156.0,18254.26
7,13081.0,17814.19
8,15311.0,17331.71
9,14088.0,15700.62


# Top 10 Stock By Quantity


In [10]:
# Top 10 Stock by Quantity
top_10_stock= pd.read_sql_query('''
    SELECT 
        StockCode,
        SUM(Quantity) as Total_Quantity
    FROM "Online_Retail.csv"
    WHERE StockCode IS NOT NULL
    GROUP BY StockCode
    order by Total_Quantity DESC
    Limit 10''',conn)

top_10_stock

,StockCode,Total_Quantity
0,21212,14994
1,84879,13641
2,85099B,12053
3,85123A,10722
4,20725,9695
5,23203,8876
6,22197,8808
7,22961,8627
8,21977,8527
9,84991,8396


# Country by Revenue

In [11]:
#  country by revenue

con_rev=pd.read_sql_query('''
    SELECT Country,Sum(Quantity * UnitPrice) as Revenue
    FROM "Online_Retail.csv"
    where CustomerID IS NOT NULL
    GROUP BY Country
    Order by Revenue DESC
    LIMIT 10;''',conn)

con_rev


,Country,Revenue
0,United Kingdom,3658927.554
1,Germany,135757.940
2,France,121154.050
3,EIRE,104155.000
4,Spain,29637.820
5,Switzerland,29636.660
6,Belgium,27651.080
7,Portugal,20319.230
8,Norway,17961.580
9,Netherlands,15563.900


# Monthly Trends

In [12]:
# Monthly revenue
pd.read_sql_query("""
    SELECT strftime('%m', InvoiceDate) AS month, 
           SUM(Quantity * UnitPrice) AS monthly_revenue
    FROM "Online_Retail.csv"
    WHERE CustomerID IS NOT NULL
    GROUP BY month
    ORDER BY month ;
""", conn)


,month,monthly_revenue
0,01,229972.030
1,02,217831.820
2,03,283486.820
3,04,240149.641
4,05,323050.430
5,06,287457.140
6,07,286325.401
7,08,303152.750
8,09,478050.402
9,10,520304.950


# Reutrns & Anomalies

In [13]:
# Return the number of negative Quantity within dataframe (Quantity < 0)
neg_quantity= pd.read_sql_query('''
    SELECT 
        count(*) as return_count
    From "Online_Retail.csv"
    Where Quantity<0;
''',conn)

neg_quantity


,return_count
0,6211


In [14]:
# Total negative unit price 
neg_unitprice=pd.read_sql_query("""
    SELECT 
        count(*) as total_neg_unit_price
    FROM "Online_Retail.csv"
    WHERE UnitPrice <= 0
""", conn)

neg_unitprice

,total_neg_unit_price
0,31


# Data Cleaning

In [15]:
# creating a Final Dataframe
myntra_sales_summary = pd.read_sql_query('''
    SELECT *,
    sum(Quantity) as Total_Quantity,
    sum(Quantity * UnitPrice) as Total_Revenue
    From "Online_Retail.csv"
    where Quantity > 0 and UnitPrice > 0 and CustomerID is NOT NULL
    group by InvoiceNo,StockCode,Description,CustomerID,Country
    order by Total_Revenue 
''',conn
)

In [16]:
myntra_sales_summary.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 328610 entries, 0 to 328609
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   InvoiceNo       328610 non-null  object 
 1   StockCode       328610 non-null  object 
 2   Description     328610 non-null  object 
 3   Quantity        328610 non-null  int64  
 4   InvoiceDate     328610 non-null  object 
 5   UnitPrice       328610 non-null  float64
 6   CustomerID      328610 non-null  float64
 7   Country         328610 non-null  object 
 8   Total_Quantity  328610 non-null  int64  
 9   Total_Revenue   328610 non-null  float64
dtypes: float64(3), int64(2), object(5)
memory usage: 25.1+ MB


In [17]:
# describe
myntra_sales_summary.describe()

,Quantity,UnitPrice,CustomerID,Total_Quantity,Total_Revenue
count,328610.000000,328610.000000,328610.000000,328610.000000,328610.000000
mean,7.607742,2.200017,15312.827607,7.694005,13.142821
std,6.794491,1.548488,1711.670852,6.822413,13.280010
min,1.000000,0.001000,12347.000000,1.000000,0.001000
25%,2.000000,1.250000,13983.000000,2.000000,4.200000
50%,6.000000,1.650000,15215.000000,6.000000,10.200000
75%,12.000000,2.950000,16813.000000,12.000000,17.400000
max,27.000000,7.500000,18287.000000,96.000000,180.000000


**Insights:**
* The average quantity purchased per transaction is about 7.4 units, with a standard deviation of 6.9, indicating moderate variability in purchase size.
* The unit price averages around £2.21, ranging from £0 to £7.50, suggesting most products are low-cost items, likely everyday consumer goods.
* The minimum quantity is -13, and minimum revenue is -£81, which are not realistic in standard retail settings. This implies returns, refunds, or possible data entry errors.
* Similarly, a unit price of £0 could indicate free items, discounts, or misentries, which may require cleansing for accurate analysis.
* The average revenue per transaction is £12.75, but the standard deviation is high (£13.51), and the maximum revenue is £180, combined with the 75th percentile being only £17.40, suggests that while most transactions are low to moderate in value, a few high-revenue purchases skew the distribution, possibly from bulk purchases or high-value items.



In [18]:
myntra_sales_summary.isnull().sum()

InvoiceNo         0
StockCode         0
Description       0
Quantity          0
InvoiceDate       0
UnitPrice         0
CustomerID        0
Country           0
Total_Quantity    0
Total_Revenue     0
dtype: int64

In [19]:
myntra_sales_summary['StockCode'].unique()

array(['PADS', 'BANK CHARGES', '16216', ..., '23049', '23052', '23066'],
      dtype=object)

In [20]:
myntra_sales_summary['Description'].unique()

array(['PADS TO MATCH ALL CUSHIONS', 'Bank Charges',
       'LETTER SHAPE PENCIL SHARPENER', ..., 'RECYCLED ACAPULCO MAT RED',
       'RECYCLED ACAPULCO MAT TURQUOISE', 'SMALL DECO JEWELLERY STAND'],
      dtype=object)

In [21]:
# removing anomalies in Quantity and UnitPrice columns
myntra_sales_summary = myntra_sales_summary[(myntra_sales_summary['Quantity'] > 0) & (myntra_sales_summary['UnitPrice'] > 0)]
myntra_sales_summary.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Total_Quantity,Total_Revenue
0,550193,PADS,PADS TO MATCH ALL CUSHIONS,1,2011-04-15 09:27:00,0.001,13952.0,United Kingdom,1,0.001
1,561226,PADS,PADS TO MATCH ALL CUSHIONS,1,2011-07-26 10:13:00,0.001,15618.0,United Kingdom,1,0.001
2,568200,PADS,PADS TO MATCH ALL CUSHIONS,1,2011-09-25 14:58:00,0.001,16198.0,United Kingdom,1,0.001
3,568375,BANK CHARGES,Bank Charges,1,2011-09-26 17:01:00,0.001,13405.0,United Kingdom,1,0.001
4,578757,16216,LETTER SHAPE PENCIL SHARPENER,1,2011-11-25 11:41:00,0.060,12748.0,United Kingdom,1,0.060


## Datatype Conversion

In [22]:
# Convert InvoiceDate to datetime datatype
myntra_sales_summary['InvoiceDate'] = pd.to_datetime(myntra_sales_summary['InvoiceDate'])

# Convert CustomerID to string or nullable integer
myntra_sales_summary['CustomerID'] = myntra_sales_summary['CustomerID'].astype(str)  

# convert the  value inside the 'Description' column into lower 
myntra_sales_summary['Description']=myntra_sales_summary['Description'].str.lower().str.strip()

myntra_sales_summary['Quantity']=pd.to_numeric(myntra_sales_summary['Quantity'])
# replace the space with '_' in the columns
myntra_sales_summary['Description']=myntra_sales_summary['Description'].str.replace(' ','_')

# extract the Year,Month,Quarter,Weekend,InVoiceHour,DayOfWeek,IsWeekend,PartOfDay  from the 'InvoiceDate' Column
myntra_sales_summary['Year']=myntra_sales_summary['InvoiceDate'].dt.year.astype("object")
myntra_sales_summary['Month']=myntra_sales_summary['InvoiceDate'].dt.month_name().astype("object")
myntra_sales_summary['Quarter']=myntra_sales_summary['InvoiceDate'].dt.quarter.astype("object")
myntra_sales_summary['Weekend']=myntra_sales_summary['InvoiceDate'].dt.weekday.astype("object")
myntra_sales_summary['InvoiceHour'] = myntra_sales_summary['InvoiceDate'].dt.hour.astype("object")
myntra_sales_summary['DayOfWeek']=myntra_sales_summary['InvoiceDate'].dt.dayofweek.astype("object")
myntra_sales_summary['IsWeekend'] = myntra_sales_summary['DayOfWeek'].isin([5, 6]).astype("bool")
myntra_sales_summary['PartOfDay']=pd.cut(myntra_sales_summary['InvoiceHour'],
                                         bins=[0,6,8,12,24],
                                         labels=['Night','Morning','Afternoon','Evening'],
                                         right=False,include_lowest=True)


# Feature Engineering 

In [23]:
# Created a New Colum "Average_Price_Per_Item"
myntra_sales_summary['Avg_Price_Per_Item'] = round(myntra_sales_summary['Total_Revenue'] / myntra_sales_summary['Total_Quantity'],3)

# Created a New column "Item_Per_Invoice"
myntra_sales_summary['Items_Per_Invoice'] = round(myntra_sales_summary.groupby('InvoiceNo')['Quantity'].transform('sum'),3)

# Drop Duplicates

In [24]:
myntra_sales_summary=myntra_sales_summary.drop_duplicates()

myntra_sales_summary['Avg_Price_Per_Item'].value_counts()

Avg_Price_Per_Item
1.250    42693
1.650    34411
2.950    25738
0.850    23881
0.420    19250
         ...  
4.333        1
2.466        1
6.360        1
3.850        1
6.980        1
Name: count, Length: 224, dtype: int64

In [25]:
myntra_sales_summary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 328610 entries, 0 to 328609
Data columns (total 20 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   InvoiceNo           328610 non-null  object        
 1   StockCode           328610 non-null  object        
 2   Description         328610 non-null  object        
 3   Quantity            328610 non-null  int64         
 4   InvoiceDate         328610 non-null  datetime64[ns]
 5   UnitPrice           328610 non-null  float64       
 6   CustomerID          328610 non-null  object        
 7   Country             328610 non-null  object        
 8   Total_Quantity      328610 non-null  int64         
 9   Total_Revenue       328610 non-null  float64       
 10  Year                328610 non-null  object        
 11  Month               328610 non-null  object        
 12  Quarter             328610 non-null  object        
 13  Weekend             328610 no

## Save Cleaned Data into Databse

In [26]:
cursor = conn.cursor()

In [27]:
# drop table 
cursor.execute(''' Drop table myntra_sales_summary''')

In [28]:
cursor.execute ('''CREATE TABLE myntra_sales_summary (
    InvoiceNo          VARCHAR(100),
    StockCode           VARCHAR(100),
    Description         TEXT,
    Quantity            INT,
    InvoiceDate         DATETIME,
    UnitPrice           FLOAT,
    CustomerID          VARCHAR(50),
    Country             VARCHAR(50),
    Total_Revenue       FLOAT,
    Year                VARCHAR(50),
    Month               VARCHAR(50),
    Quarter             VARCHAR(50),
    Weekend             VARCHAR(50),
    InvoiceHour         Time,
    DayOfWeek           VARCHAR(50),
    IsWeekend           BOOLEAN,
    PartOfDay           VARCHAR(50),
    Avg_Price_Per_Item  FLOAT,
    Items_Per_Invoice   INT
);
''')

In [29]:
myntra_sales_summary.to_sql('myntra_sales_summary',conn,if_exists='replace',index=False)

328610

# Save Clean Dataset into an csv file.

In [30]:
myntra_sales_summary.to_csv('myntra_sales_summary.csv', header=True , index=False , mode='w')